# GPT promt mvp

In [5]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('API_KEY')

La idea es pasar de lenguaje natural al un dato estructurado para hacer la query.

In [57]:
import json

examples = json.load(open('examples.json'))['data']

In [58]:
qa_text = ""
for example in examples:
  if example['train']:
    qa_text += f"Q: {str(example['query'])}\n"
    qa_text += f"A: {json.dumps(example['answer'])}"
    qa_text += '\n'
    

In [59]:
prompt = lambda text: f"""
Te pasaré una consulta entre los simbolos <>. Tu tarea es:
1.- Reconocer si el texto es una consulta relacionada a desastres climaticos o centros asistenciales. Si ni lo es responder "null".
2.- Parsear la consulta y retornarn SOLO un JSON con los siguientes datos:
- ubicacion: string | null
- tipo_de_desastre: (incendio, tsunami, terremoto)
- centro_asistencial: (bomberos, policia, hospital, areas_verdes)
La respuesta deberá se, o null si no es relacionada, o el json.

Algunos ejemplos de QA:
{qa_text}
<{text}>
"""

In [60]:
def get_completion_gpt3_5(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [61]:
import json

examples = json.load(open('examples.json'))['data']

In [62]:
for example in examples:
  if example['train'] == False:
    print(example['query'])
    print(get_completion_gpt3_5(prompt(example['query'])))
    print('\n')

a cuanto esta el hospital de chillan
{"ubicacion": "chillan", "tipo_de_desastre": null, "centro_asistencial": "hospital"}


como hacer un print en python
{"ubicacion": null, "tipo_de_desastre": null, "centro_asistencial": null}


En Valparaiso peligran las areas verdes a fin de año con los incendios? 
{"ubicacion": "Valparaiso", "tipo_de_desastre": "incendio", "centro_asistencial": "areas_verdes"}


Hospitales en angol
{"ubicacion": "angol", "tipo_de_desastre": null, "centro_asistencial": "hospital"}


Incendios en la antartica
{"ubicacion": "antartica", "tipo_de_desastre": "incendio", "centro_asistencial": null}


